In [1]:
import pandas as pd

dis = pd.read_csv('D:\python_code\work2\data_used\dis_before_baseline_merged.csv') #发生在touchscreen0.0之前的疾病
touch_data = pd.read_csv('D:\python_code\work2\single data\data_processed\Touchscreen_85.csv')

In [2]:
touch_data = touch_data.mask(touch_data < 0)

missing_counts = touch_data.isnull().sum(axis=1)
threshold = 0.2 * touch_data.shape[1]

touch = touch_data[missing_counts < threshold]
touch.shape

(482080, 86)

In [3]:
mental_health = dis.iloc[:, 1:12]
dis['mental'] = mental_health.sum(axis=1)
physical_health = dis.iloc[:, 12:-1]
dis['physical'] = physical_health.sum(axis=1)

In [5]:
dis['Multi'] = dis[['mental','physical']].sum(axis=1)

In [6]:
sum = dis[['eid', 'mental', 'physical', 'Multi']].copy()
data = pd.merge(touch, sum, on='eid') #touch
data.shape

(482057, 89)

In [7]:
import pandas as pd

gmv_data = pd.read_csv('D:/python_code/work2/brain_data_structure/GMV_2.0_dropna.csv')
fa_data = pd.read_csv('D:/python_code/work2/brain_data_structure/FA_2.0_dropna.csv')
gmv_eid = gmv_data['eid']
fa_eid = fa_data['eid']

protein_data = pd.read_csv('D:/python_code/work2/protein/protein_ins0.csv')
protein_eid = protein_data['eid']

combined_eid = pd.concat([gmv_eid, fa_eid, protein_eid]).drop_duplicates()

In [8]:
trian_data = data[~data['eid'].isin(combined_eid)].copy()
validation_data = data[data['eid'].isin(combined_eid)].copy()

print('trian and test data:', trian_data.shape)
print('validation data:', validation_data.shape)

trian and test data: (391193, 89)
validation data: (90864, 89)


In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(trian_data, test_size=0.3, random_state=29)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.linear_model import BayesianRidge

features = x_train.iloc[:, 1:-3].values
variant = x_train[['eid','mental','physical','Multi']].copy()
variant_reset = variant.reset_index(drop=True)

# -------------------------
# Bayesian Ridge
# -------------------------
class BayesianRidgeImputer:
    def __init__(self):
        self.model = BayesianRidge()
        self.categorical_indices = []  # To store indices of categorical variables
        self.continuous_indices = []   # To store indices of continuous variables

    def _identify_variable_types(self, X):
        """Identify categorical and continuous variables based on the data."""
        for i in range(X.shape[1]):
            if np.all(np.isnan(X[:, i]) | (X[:, i] == np.floor(X[:, i]))):
                self.categorical_indices.append(i)
            else:
                self.continuous_indices.append(i)

    def fit_transform(self, X):
        mask = np.isnan(X)  # Mask for missing values
        X_filled = X.copy()
        self._identify_variable_types(X)

        for i in range(X.shape[1]):
            if mask[:, i].any():
                # Median init
                median_imputer = SimpleImputer(strategy='median')
                X_filled[:, i] = median_imputer.fit_transform(X[:, i].reshape(-1, 1)).ravel()

                temp_X = X_filled.copy()
                temp_X[:, [j for j in range(X.shape[1]) if j != i]] = SimpleImputer(strategy='median').fit_transform(
                    temp_X[:, [j for j in range(X.shape[1]) if j != i]]
                )

                not_missing_idx = ~mask[:, i]
                missing_idx = mask[:, i]

                if not_missing_idx.sum() > 0:
                    self.model.fit(temp_X[not_missing_idx][:, [j for j in range(X.shape[1]) if j != i]], 
                                   X_filled[not_missing_idx, i])

                    predicted_values = self.model.predict(temp_X[missing_idx][:, [j for j in range(X.shape[1]) if j != i]])

                    if i in self.categorical_indices:
                        X_filled[missing_idx, i] = np.round(predicted_values)
                    else:
                        X_filled[missing_idx, i] = np.maximum(0, predicted_values)

        return X_filled

# -------------------------
# Complete Case Analysis
# -------------------------
mask_complete = ~np.isnan(features).any(axis=1)
features_cca = features[mask_complete]
variant_cca = variant_reset.loc[mask_complete].reset_index(drop=True)

data_cca_full = pd.concat([variant_cca, pd.DataFrame(features_cca, columns=x_train.columns[1:-3]).reset_index(drop=True)], axis=1)
data_cca_full.to_csv('train_data_CCA.csv', index=False)
print("Complete Case Analysis")

# -------------------------
# Multiple Imputation
# -------------------------
mi_imputer = IterativeImputer(estimator=BayesianRidge(), max_iter=10, random_state=0)
features_mi = mi_imputer.fit_transform(features)
data_mi = pd.DataFrame(features_mi, columns=x_train.columns[1:-3])
data_mi_full = pd.concat([variant_reset, data_mi.reset_index(drop=True)], axis=1)
data_mi_full.to_csv('train_data_MI.csv', index=False)
print("Multiple Imputation")